In [1]:
from entity_network import EntityNetwork
import re
import numpy as np
import pandas as pd
import collections
from sklearn import metrics
import tensorflow as tf
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from tqdm import tqdm
import time

In [2]:
permulaan = [
    'bel',
    'se',
    'ter',
    'men',
    'meng',
    'mem',
    'memper',
    'di',
    'pe',
    'me',
    'ke',
    'ber',
    'pen',
    'per',
]

hujung = ['kan', 'kah', 'lah', 'tah', 'nya', 'an', 'wan', 'wati', 'ita']

def naive_stemmer(word):
    assert isinstance(word, str), 'input must be a string'
    hujung_result = re.findall(r'^(.*?)(%s)$' % ('|'.join(hujung)), word)
    word = hujung_result[0][0] if len(hujung_result) else word
    permulaan_result = re.findall(r'^(.*?)(%s)' % ('|'.join(permulaan[::-1])), word)
    permulaan_result.extend(re.findall(r'^(.*?)(%s)' % ('|'.join(permulaan)), word))
    mula = permulaan_result if len(permulaan_result) else ''
    if len(mula):
        mula = mula[1][1] if len(mula[1][1]) > len(mula[0][1]) else mula[0][1]
    return word.replace(mula, '')

In [3]:
def build_dataset(words, n_words):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 3)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary


def classification_textcleaning(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    string = unidecode(string).replace('.', ' . ').replace(',', ' , ')
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = ' '.join(
        [i for i in re.findall('[\\w\']+|[;:\-\(\)&.,!?"]', string) if len(i)]
    )
    string = string.lower().split()
    string = [(naive_stemmer(word), word) for word in string]
    return (
        ' '.join([word[0] for word in string if len(word[0]) > 1]),
        ' '.join([word[1] for word in string if len(word[0]) > 1]),
    )


def str_idx(corpus, dic, maxlen, UNK = 3):
    X = np.zeros((len(corpus), maxlen))
    for i in range(len(corpus)):
        for no, k in enumerate(corpus[i].split()[:maxlen][::-1]):
            X[i, -1 - no] = dic.get(k,UNK)
    return X

In [4]:
with open('subjectivity-negative-translated.txt','r') as fopen:
    texts = fopen.read().split('\n')
labels = [0] * len(texts)

with open('subjectivity-positive-translated.txt','r') as fopen:
    positive_texts = fopen.read().split('\n')
labels += [1] * len(positive_texts)
texts += positive_texts

assert len(labels) == len(texts)

In [5]:
for i in range(len(texts)):
    texts[i] = classification_textcleaning(texts[i])[0]

In [6]:
concat = ' '.join(texts).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 13222
Most common words [('yang', 11804), ('untuk', 3879), ('tidak', 2898), ('deng', 2827), ('ada', 2294), ('dalam', 2193)]
Sample data [10, 68, 13, 27, 56, 53, 11, 389, 34, 183] ['filem', 'mula', 'pada', 'masa', 'lalu', 'mana', 'orang', 'budak', 'lelaki', 'nama']


In [7]:
size_layer = 256
num_layers = 2
embedded_size = 256
dimension_output = len(np.unique(labels))
learning_rate = 1e-3
maxlen = 80
batch_size = 128
decay_step = 1e4
decay_rate = 1.0
story_len = 1

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

model = EntityNetwork(
    dimension_output,
    learning_rate,
    decay_step,
    decay_rate,
    maxlen,
    story_len,
    len(dictionary),
    embedded_size,
    size_layer
)

sess.run(tf.global_variables_initializer())

In [9]:
vectors = str_idx(texts, dictionary, maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(
    vectors, labels, test_size = 0.2
)

In [10]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'learning_rate' not in n.name
        and 'OptimizeLoss' not in n.name
        and 'Global_Step' not in n.name
        and 'Epoch_Step' not in n.name
    ]
)
strings.split(',')

['Placeholder_story',
 'Placeholder_question',
 'Placeholder_input_y',
 'Placeholder_dropout_keep_prob',
 'output_module/H',
 'output_module/R',
 'output_module/y_bias',
 'output_module/b_projection',
 'output_module/h_u_bias',
 'dynamic_memory/U',
 'dynamic_memory/V',
 'dynamic_memory/W',
 'dynamic_memory/h_bias',
 'dynamic_memory/h2_bias',
 'embedding_projection/Embedding',
 'story_mask',
 'query_mask',
 'hidden_states',
 'keys',
 'h_candidate0/alpha',
 'query_add_hidden/alpha',
 'logits']

In [11]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'entity_network/model.ckpt')

'entity_network/model.ckpt'

In [12]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.answer_single: batch_y,
                model.query: batch_x,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.answer_single: batch_y,
                model.query: batch_x,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   8%|▊         | 5/63 [00:00<00:01, 42.20it/s, accuracy=0.859, cost=1.08]

epoch: 0, pass acc: 0.000000, current acc: 0.891985
time taken: 1.7730424404144287
epoch: 0, training loss: 2.012584, training acc: 0.733433, valid loss: 1.191522, valid acc: 0.891985



train minibatch loop:   8%|▊         | 5/63 [00:00<00:01, 42.19it/s, accuracy=0.922, cost=0.675]

epoch: 1, pass acc: 0.891985, current acc: 0.898508
time taken: 1.5795412063598633
epoch: 1, training loss: 0.891616, training acc: 0.912285, valid loss: 0.768697, valid acc: 0.898508



train minibatch loop:   8%|▊         | 5/63 [00:00<00:01, 42.54it/s, accuracy=0.93, cost=0.502] 

...

time taken: 1.5752253532409668
epoch: 3, training loss: 0.419431, training acc: 0.965868, valid loss: 0.563405, valid acc: 0.879936



train minibatch loop:   8%|▊         | 5/63 [00:00<00:01, 41.81it/s, accuracy=0.961, cost=0.335]

time taken: 1.575713872909546
epoch: 4, training loss: 0.348387, training acc: 0.968879, valid loss: 0.650409, valid acc: 0.829382



test minibatch loop: 100%|██████████| 16/16 [00:00<00:00, 180.95it/s, accuracy=0.89, cost=0.453]

time taken: 1.5731379985809326
epoch: 5, training loss: 0.298165, training acc: 0.974652, valid loss: 0.461024, valid acc: 0.904653

break epoch:6



In [13]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    batch_x_expand = np.expand_dims(batch_x,axis = 1)
    predict_Y += np.argmax(
        sess.run(
            model.logits,
            feed_dict = {
                model.answer_single: batch_y,
                model.query: batch_x,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 16/16 [00:00<00:00, 178.82it/s]


In [14]:
from sklearn import metrics
print(metrics.classification_report(real_Y, predict_Y, target_names = ['negative','positive']))

              precision    recall  f1-score   support

    negative       0.89      0.87      0.88       984
    positive       0.87      0.89      0.88      1009

   micro avg       0.88      0.88      0.88      1993
   macro avg       0.88      0.88      0.88      1993
weighted avg       0.88      0.88      0.88      1993



In [15]:
text = classification_textcleaning(
    'kerajaan sebenarnya sangat bencikan rakyatnya, minyak naik dan segalanya'
)
new_vector = str_idx([text[0]], dictionary, maxlen)
batch_x_expand = np.expand_dims(new_vector,axis = 1)
sess.run(
            tf.nn.softmax(model.logits),
            feed_dict = {
                model.query: new_vector,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        )

array([[0.2109821, 0.7890179]], dtype=float32)

In [16]:
saver.save(sess, 'entity_network/model.ckpt')

'entity_network/model.ckpt'

In [17]:
import json
with open('entity-network-subjective.json','w') as fopen:
    fopen.write(json.dumps({'dictionary':dictionary,'reverse_dictionary':rev_dictionary}))

In [18]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [19]:
freeze_graph('entity_network', strings)

INFO:tensorflow:Restoring parameters from entity_network/model.ckpt
INFO:tensorflow:Froze 17 variables.
INFO:tensorflow:Converted 17 variables to const ops.
139 ops in the final graph.


In [20]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [21]:
g = load_graph('entity_network/frozen_model.pb')

In [22]:
Placeholder_story = g.get_tensor_by_name('import/Placeholder_story:0')
Placeholder_question = g.get_tensor_by_name('import/Placeholder_question:0')
Placeholder_dropout_keep_prob = g.get_tensor_by_name(
    'import/Placeholder_dropout_keep_prob:0'
)
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)
test_sess.run(
    tf.nn.softmax(logits),
    feed_dict = {
        Placeholder_question: new_vector,
        Placeholder_story: batch_x_expand,
        Placeholder_dropout_keep_prob: 1.0,
    },
)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([[0.2109821, 0.7890179]], dtype=float32)